In [1]:
%matplotlib inline

from itertools import starmap, chain
from collections import Counter

import numpy as np
import ggplot
from fn import F

from chempred.chemdner import read_abstracts, read_annotations, pair, annotate_abstract
from chempred import preprocessing as pp
import prototype

/home/ilia/.venvs/py3/lib/python3.5/site-packages/ggplot/utils.py:81: FutureWarning: pandas.tslib is deprecated and will be removed in a future version.
You can access Timestamp as pandas.Timestamp
  pd.tslib.Timestamp,
/home/ilia/.venvs/py3/lib/python3.5/site-packages/ggplot/stats/smoothers.py:4: FutureWarning: The pandas.lib module is deprecated and will be removed in a future version. These are private functions and can be accessed from pandas._libs.lib instead
  from pandas.lib import Timestamp
/home/ilia/.venvs/py3/lib/python3.5/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools
Using TensorFlow backend.


In [2]:
window = 5

abstracts_train = read_abstracts("chemdner_corpus/training.abstracts.txt")
anno_train = read_annotations("chemdner_corpus/training.annotations.txt")
pairs_train = list(zip(abstracts_train, pair(abstracts_train, anno_train)))
annotated_abstracts_train = list(starmap(F(annotate_abstract, guided=True), pairs_train))

abstracts_test = read_abstracts("chemdner_corpus/evaluation.abstracts.txt")
anno_test = read_annotations("chemdner_corpus/evaluation.annotations.txt")
pairs_test = list(zip(abstracts_test, pair(abstracts_test, anno_test)))
annotated_abstracts_test = list(starmap(F(annotate_abstract, guided=True), pairs_test))

pos_cls = Counter(
    r[-1] for r in chain.from_iterable(anno for _, anno in anno_train + anno_test)
)

# TODO rename generate_training_samples
samples_train, cls_train = pp.join_tokens_in_samples(
    *pp.generate_training_samples(annotated_abstracts_train, pos_cls, window, False))
samples_test, cls_test = pp.join_tokens_in_samples(
    *pp.generate_training_samples(annotated_abstracts_test, pos_cls, window, False))

max_sample_len = max(map(len, samples_train+samples_test))

padded_samples_train, padded_cls_train, masks_train = pp.pad(samples_train, cls_train, 
                                                             max_sample_len)
padded_samples_test, padded_cls_test, masks_test = pp.pad(samples_test, cls_test, 
                                                          max_sample_len)

onehot_samples_train, onehot_cls_train = map(
    pp.encode_one_hot, [padded_samples_train, padded_cls_train])
masked_samples_train = pp.mask_array(onehot_samples_train, masks_train)
masked_cls_train = pp.mask_array(onehot_cls_train, masks_train)

onehot_samples_test, onehot_cls_test = map(
    pp.encode_one_hot, [padded_samples_test, padded_cls_test])
masked_samples_test = pp.mask_array(onehot_samples_test, masks_test)
masked_cls_test = pp.mask_array(onehot_cls_test, masks_test)

In [3]:
max_sample_len

635

In [8]:
# class_counts = np.vstack([np.unique(cls, return_counts=True)[1] for cls in 
#                           (padded_cls[mask] for padded_cls, mask in zip(padded_classes, masks))])
# class_counts[:10]

array([[34, 11],
       [36, 11],
       [33, 11],
       [29, 11],
       [29, 11],
       [26,  9],
       [24,  9],
       [24,  9],
       [18,  9],
       [22,  9]])

In [4]:
masked_samples_train.shape, masked_samples_test.shape

((143616, 635, 240), (123380, 635, 240))

In [5]:
from keras import backend as k
from keras import losses
from keras import models
from keras import layers
from keras import callbacks

In [ ]:
nsteps = 200
maxlen = masked_samples_train.shape[1]
nchar = masked_samples_train.shape[2]
ncls = 2
nrec = 2
batchsize = 200

lstm_inp_drop = 0.1
lstm_rec_drop = 0.1

l_in = layers.Input(shape=(maxlen, nchar), name="l_in")
l_mask = layers.Masking(mask_value=0, name="l_mask")(l_in)
l_rec = prototype.build_rec([nsteps]*nrec, [lstm_inp_drop]*nrec, [lstm_rec_drop]*nrec)(l_mask)
l_out = layers.TimeDistributed(
    layers.Dense(ncls, activation='softmax'), name="l_out")(l_rec)
model = models.Model(l_in, l_out)
model.compile(optimizer="adam", loss="binary_crossentropy", metrics = ["accuracy"])

filepath = "weights-improvement-{epoch:02d}-{val_acc:.2f}.hdf5"
checkpoint = callbacks.ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
# reduce_lr = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3,min_lr=0.0001)

callbacks_ = [checkpoint]

model.fit(masked_samples_train, masked_cls_train, batch_size=batchsize, epochs=30, verbose=1,
          validation_data=(masked_samples_test, masked_cls_test), callbacks=callbacks_)

Train on 143616 samples, validate on 123380 samples
Epoch 1/30
  2600/143616 [..............................] - ETA: 6727s - loss: 0.6857 - acc: 0.6230

In [43]:
print(test2_x_encoded[:1].argmin(axis=2))
print(net2.predict(test2_x_encoded[:1]).argmax(axis=2))

[[0 0 1 1 1 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]
[[0 0 1 1 1 1 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]]
